# Hydrograph-Py
Hydrograph-py is a hydrological Python package that provides some tools for:

1. Separation of flow time-series into peak flow and baseflow.

2. Filtering of peak flow events given a minimum event duration.

3. Calculation of peak event volumes.

4. Calculation of maximum annual peak flow and maximum annual peak event volume.

5. Extreme value analysis using GEV fitting and plotting functions.

Streamflow separation in this package is based on the principle introduced by [1]. They separated the hydrograph into “quickflow” and “delayed flow” components by arbitrarily projecting a line of constant slope from the beginning of any stream rise until it intersected the falling side of the hydrograph.

(1) J.D. Hewlett and A.R. Hibbert. Factors affecting the responseof small watersheds to precipitation in humid areas. In W.E. Sopper and H.W. Lull, editors, Forest Hydrology, 275–290. Oxford, 1967.

In [ ]:
from Hydrograph.hydrograph import sepBaseflow, filterpeaks, maxFlowVolStats

from Hydrograph.extreme_analysis import exceed, fitGEV, plotPDF, plotCDF, plotGEV

## sepBaseflow
The ```sepBaseflow``` function separates a time-series into baseflow and peakflow. Fills missing flow records by interpolation. The input and output for this function are shown below.

In [ ]:
import pandas as pd
sespe = pd.read_csv('data/sespe-raw.csv',parse_dates=['Date'])
sespe = sespe.set_index('Date')
sespe['Total runoff [m^3 s^-1]'] = sespe['Total runoff [m^3 s^-1]'].astype(float)

sespe.head(5)

In [ ]:
def sepBaseflow(x, dt, A, k=0.000546, dt_max=None, tp_min=None):
    '''
    Separate a time-series into baseflow and peakflow. Fills missing flow records by interpolation.

    -----------------------------------------------------------------------------------------------
    Input:
        x:          Pandas dataframe with Index being a pandas datetime index and 'Date' label. Dataframe should.
                    contain one column for flow data, and should be labeled 'Total runoff [m^3 s^-1]'.
        dt:         Minimum time-step interval (in minutes) for analysing the data. Minute choices are 5, 15, or 60.
        k:          Slope of the dividing line; i.e. slope that defines when peakflow event starts and baseflow separation occurs.
                    Default is 0.000546  m^3 s^-1 km^-2 h^-1 (Hewlett and Hibbert 1967).
        A:          Catchment area in km^2 upstream of point of interest.
        dt_max:     Only interpolate over maximum number of consecutive NaN defined over time period dt_max in hours.
        tp_min:     Minimum duration of runoff peak in hours to be selected as being a peak.
    -----------------------------------------------------------------------------------------------
    Returns:
        df_final:    Pandas dataframe with datetime index and the following columns:
                        dt [hour]:                       Time difference in hours between two records.
                        Total runoff [m^3 s^-1]:         Recorded flow in cumecs for that timestamp.
                        Total runoff interp. [m^3 s^-1]: Interpolated recorded flow in cumecs.
                        Baseflow [m^3 s^-1]:             Calculated baseflow in cumecs for that timestamp.
                        Peakflow [m^3 s^-1]:             Calculated peakflow in cumecs for that timestamp.
                        Peak nr.:                        Peak number in sequence. Each peakflow event (i.e. flow above baseflow) is given a unique number
                                                         if it classifies as being a peak after filtering.
                        Peakflow starts:                 Timestamp when peakflow starts (moment when runoff peak exceeds baseflow).
                        Peakflow ends:                   Timestamp when peakflow ends (moment when runoff peak itersects again with baseflow).
                        Flow volume [m^3]:               Volume of the flow between two time-steps (total volume; i.e. baseflow + peakflow).
                        Max flow [m^3 s^-1]:             Maximum flow of peak flow event.
                        Date max. flow:                  Timestamp of maximum flow of peak flow event.
                        Tp [hour]:                       Time to peak.
    '''

In [ ]:
sespe_baseflow = sepBaseflow(sespe,15,20)
sespe_baseflow.head()

In [ ]:
import plotly_express as px
fig = px.line(sespe_baseflow, x=sespe_baseflow.index, y=["Total runoff [m^3 s^-1]",'Baseflow [m^3 s^-1]'],title="Sespe River Flow")
fig.show()